In [7]:
from typing import Annotated

from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langgraph.graph import END, StateGraph
from langgraph.graph.message import add_messages
from pydantic import BaseModel


class State(BaseModel):
    messages: Annotated[list[BaseMessage], add_messages]


def echo_node(state: State) -> State:
    msg = state.messages[-1].content
    state.messages.append(AIMessage(content=f"Echo: {msg}"))
    return state


workflow = StateGraph(State)
workflow.add_node(echo_node)
workflow.set_entry_point("echo_node")
workflow.add_edge("echo_node", END)
graph = workflow.compile()

# Run with initial state
result = graph.invoke(State(messages=[HumanMessage(content="Hi!")]))
print(result)  # "Echo: Hi!"

{'messages': [HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}, id='c7f7d4c2-1a3f-437f-94a4-9de37562667c'), AIMessage(content='Echo: Hi!', additional_kwargs={}, response_metadata={}, id='52d60598-931e-4c7b-96cd-ea2affe2edd0')]}


In [6]:
for message in result["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi!
================================== Ai Message ==================================

Echo: Hi!


# Set conditional nodes

In [ ]:
from langchain.chat_models import init_chat_model
from langchain.schema import AIMessage, HumanMessage
from langgraph.graph import END, StateGraph
from langgraph.graph.message import add_messages


# Step 1: Define your state
class GraphState(dict):
    pass


# Step 2: Define a simple node that sends message to LLM
llm = init_chat_model("gpt-4.1-mini")


def chat_node(state: GraphState) -> GraphState:
    messages = state.get("messages", [])
    response = llm.invoke(messages)
    messages.append(AIMessage(content=response.content))
    return {"messages": messages}


# Step 3: Define the graph
builder = StateGraph(GraphState)
builder.add_node("chat", chat_node)

builder.set_entry_point("chat")

builder.add_edge("chat", END)

graph = builder.compile()

# Step 4: Run it
initial_state = {"messages": [HumanMessage(content="What's the capital of France?")]}

final_state = graph.invoke(initial_state)

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph import END, START, StateGraph
from langgraph.graph.message import add_messages


class State:
    messages: list[BaseMessage]


llm = init_chat_model("gpt-4o-mini")


def generate_response(state):
    return {"messages": llm.invoke(state["messages"])}


graph = StateGraph(State)

graph.add_node("generate_response", generate_response)

graph.add_edge(START, "generate_response")
graph.add_edge("generate_response", END)

graph = graph.compile()

graph.invoke({"messages": [HumanMessage(content="What is the capital of France?")]})

In [ ]:
graph.compile()

In [ ]:
graph.invoke({"messages": [HumanMessage(content="What is the capital of France?")]})